In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from bokeh.plotting import figure, output_notebook, show

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [4]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [295]:
data = np.loadtxt("Si111_40_30.xy").T
err = np.sqrt(data[1])
err = np.where(err < 2, 2, err)

In [296]:
plt.errorbar(data[0],data[1],err,label="Messdaten, mit U = 40kV \n A = 30mA und Ni-Filter")
plt.xlim([15,130])
plt.ylim([-10,28000])
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.xlabel('Winkel / $^\circ$',fontsize=32)
plt.ylabel('Counts',fontsize=32)
plt.legend(loc='best',fontsize=22)
plt.show()

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdiraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [308]:
intervall_voigt_1 = np.arange(np.where(data[0] > 25.6)[0][0], np.where(data[0] > 25.69)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [309]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1['voigt_1gamma'].set(0.001,vary=True,expr='')

In [310]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/(err_1))

In [311]:
out_1.plot()
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ')
plt.show()

In [312]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 31
    # data points      = 22
    # variables        = 4
    chi-square         = 169.680
    reduced chi-square = 9.427
[[Variables]]
    voigt_1sigma:       0.01436180 +/- 0.000631 (4.39%) (init= 0.01222012)
    voigt_1center:      25.6455563 +/- 0.000146 (0.00%) (init= 25.64408)
    voigt_1amplitude:   1105.70621 +/- 14.34691 (1.30%) (init= 1000.658)
    voigt_1fwhm:        0.05172131 +/- 0.002272 (4.39%)  == '3.6013100*voigt_1sigma'
    voigt_1gamma:       0.00846625 +/- 0.000803 (9.49%) (init= 0.001)
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.967 
    C(voigt_1amplitude, voigt_1gamma)  =  0.868 
    C(voigt_1sigma, voigt_1amplitude)  = -0.828 


### Mulit-Voigt

In [313]:
intervall_voigt_2 = np.arange(np.where(data[0] > 28.39)[0][0], np.where(data[0] > 28.54)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 28.4 < data[0])[0][0],np.where( data[0] > 28.452)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 28.452 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [314]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))
pars_2['voigt_2gamma'].set(0.000001,vary=True,expr='')
pars_2['voigt_3gamma'].set(0.000001,vary=True,expr='')
pars_2['voigt_2amplitude'].set(1000)
pars_2['voigt_3amplitude'].set(1000)

In [315]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2))

In [316]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ')
plt.xlim([28,29])
plt.show()

In [317]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 187
    # data points      = 36
    # variables        = 8
    chi-square         = 2097.715
    reduced chi-square = 74.918
[[Variables]]
    voigt_2sigma:       0.02559636 +/- 0.003664 (14.32%) (init= 0.01222012)
    voigt_2amplitude:   2225.75595 +/- 342.9151 (15.41%) (init= 1000)
    voigt_2center:      28.4346989 +/- 0.001619 (0.01%) (init= 28.43068)
    voigt_2fwhm:        0.09218045 +/- 0.013197 (14.32%)  == '3.6013100*voigt_2sigma'
    voigt_2gamma:      -0.00374697 +/- 0.007768 (207.32%) (init= 1e-06)
    voigt_3sigma:       0.02583595 +/- 0.008128 (31.46%) (init= 0.02308244)
    voigt_3amplitude:   1113.47325 +/- 522.0124 (46.88%) (init= 1000)
    voigt_3center:      28.5085611 +/- 0.000917 (0.00%) (init= 28.4948)
    voigt_3fwhm:        0.09304327 +/- 0.029272 (31.46%)  == '3.6013100*voigt_3sigma'
    voigt_3gamma:      -0.01653587 +/- 0.023415 (141.61%

In [318]:
plt.subplot(121)
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,label="Messdaten")
plt.plot(np.arange(25.59,25.70,0.0001),out_1.eval(x=np.arange(25.59,25.70,0.0001)),label="Multi-Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.title('Fit für den einzelnen Peak',fontsize=42)


plt.subplot(122)
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,label="Messdaten")
plt.plot(np.arange(28.39,28.545,0.0001),out_2.eval(x=np.arange(28.39,28.545,0.0001)),label="Multi-Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.title('Fit für die überlagerten Peaks',fontsize=42)

plt.show()

# Plots der Spektren für die ersten 4 Aufnahme

In [ ]:
data_spektrum = np.loadtxt("Si111_40_30_Ni.xy").T

# Ni-Verhältnis

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

In [ ]:
data_ni_o = np.loadtxt("Si111_40_30.xy").T
data_ni_m = np.loadtxt("Si111_40_30_Ni.xy").T

In [ ]:
plt.subplot(212)
plt.errorbar(data_ni_m[0],data_ni_m[1],np.sqrt(data_ni_m[1]),fmt='. b',label="Messdaten mit Filter")
plt.xlim([28,29])
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.subplot(211)
plt.errorbar(data_ni_o[0],data_ni_o[1],np.sqrt(data_ni_o[1]),fmt='. r',label="Messdaten ohne Filter")
plt.xlim([40,55])
plt.ylim([0,20])
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.show()

In [ ]:
plt.errorbar(data_ni_o[0],data_ni_o[1],np.sqrt(data_ni_o[1]),fmt='. b',label="Messdaten ohne Filter")
plt.errorbar(data_ni_m[0],data_ni_m[1],np.sqrt(data_ni_m[1]),fmt='. r',label="Messdaten mit Filter")
plt.xticks(fontsize=22)
plt.yticks(fontsize=22)
plt.legend(loc='best', fontsize=32)

plt.show()

In [ ]:
10 * np.log(1500/(1500-1046))

In [ ]:
from uncertainties import ufloat
from uncertainties.umath import log as lg
from uncertainties.umath import sin as si

y = ufloat(1500,41)
x = ufloat(1814,613)

print(10*lg(x/abs(x-y)))

In [ ]:
x = ufloat(313,2)
y = ufloat(771,6)

print(10*lg(y/abs(x-y)))

In [ ]:
x = ufloat(837,40)
y = ufloat(1046,273)

print(10*lg(y/abs(x-y)))

In [ ]:
x = ufloat(157,2)
y = ufloat(363,4)

print(10*lg(y/abs(x-y)))

$\huge E = \frac{nhc}{2 \cdot d \cdot sin(\Theta)}$

In [ ]:
hc = 12.39841875
d_si = 3.1355

In [ ]:
def E(theta,n=1):
    return (n*hc)/(2*d_si*si((np.pi * theta)/180))

In [ ]:
(hc)/(2*d_si*si((np.pi * ufloat(25.6390/2,0.007))/180))/8905.3

In [ ]:
E(ufloat(25.6390/2,0.007))

In [ ]:
E(ufloat(28.4278/2,0.007))/8047,8

In [ ]:
E(ufloat(28.4996/2,0.007))/8027,8